In [ ]:
from pyspark.sql import SparkSession #Importamos lo necesario
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('CADS').getOrCreate() #Abrimos Sesión en Spark

In [ ]:
test = spark.read.json("../common/data/test.json") #Abrimos el archivo Test

In [ ]:
test = test.drop('helpful','summary','unixReviewTime','reviewerName') #Quitamos las columnas que no sirven
test = test.selectExpr('asin as productId', 'overall as rating', 'reviewerID as reviewerId') #Renombramos las columnas
test = test.fillna(value='',subset='reviewText') #Llenamos los textos nulos
test = test.withColumn("rating",test["rating"].cast("Integer")) #Convertimos rating a entero
test = test.withColumn("reviewTime",translate("reviewTime",",","")) #Corregimos el formato de la fecha
test = test.withColumn("reviewTime",translate("reviewTime"," ","-")) #Reformateamos la fecha
test = test.withColumn("reviewTime",to_date(col("reviewTime"),"MM-d-yyyy")) #Convertimos la fecha a fecha

In [ ]:
test.write.json('../common/data/test_etl') #Guardamos el DataFrame

In [ ]:
test.printSchema() #df.info()
test.show() #df.head(20)

In [ ]:
Dict_Null = {col:test.filter(test[col].isNull()).count() for col in test.columns} #Armamos un diccionario con las cuentas de nulos
Dict_Null #Vemos la cuenta de nulos